# Paddle给我来个表情

**随着新年的到来，在微信发红包肯定是必不可少的。在发红包，加个GIF上去，贼有逼格。但是怎么加GIF呢？Paddle直接帮你将一张照片变为GIF**

![](https://ai-studio-static-online.cdn.bcebos.com/73891cae9c7243808e81fd55c6fbe54b920747640fbb429791ee045ce8b43e27)
![](https://ai-studio-static-online.cdn.bcebos.com/47cc34052dcc4046a166e65b8463fdc83a65394957924e4b93d3628678390a25)



**在运行Fastspeech文字转语音时，可能会炸显存，取决于文本量，建议用32GB版本的V100来运行此项目，查看方法，请到我另一个项目中查看哦，[传送门](https://aistudio.baidu.com/aistudio/projectdetail/1499994)**

# 1、安装Parakeet
备注：不要从GitHub直接下载，直接下载的是release/v0.2，在0.2版本没有fastspeech的example，所以要下载的版本是**release/v0.1**

In [ ]:
!tar -xf /home/aistudio/Parakeet.tar
%cd /home/aistudio/Parakeet
!pip install -e .
%cd ..

/home/aistudio/Parakeet
Looking in indexes: https://mirror.baidu.com/pypi/simple/
Obtaining file:///home/aistudio/Parakeet
     |████████████████████████████████| 3.7MB 14.8MB/s eta 0:00:01
  Found existing installation: numba 0.48.0
    Uninstalling numba-0.48.0:
      Successfully uninstalled numba-0.48.0
  Found existing installation: parakeet 0.0.0
    Uninstalling parakeet-0.0.0:
      Successfully uninstalled parakeet-0.0.0
  Running setup.py develop for parakeet
/home/aistudio


## 立即重启环境，Parakeet才能使用，否则后面程序会报错的
![](https://ai-studio-static-online.cdn.bcebos.com/93c2e8d814804671a420a0422a11ad80fd814764e64d4e94a90ab2686edbb822)



# 2、清除上次运行的临时文件
备注：有#的一定要从**终端**删除，在notebook是无法删除的。

![](https://ai-studio-static-online.cdn.bcebos.com/8f6e99879cfc4ea5addf74bbd9e372071cf60fde4388455097ed46ccf6428360)

In [ ]:
#rm -rf Parakeet/examples/fastspeech/waveflow_article3.wav Parakeet/examples/fastspeech/waveflow_article3.txt Parakeet/examples/fastspeech/synthesis/samples/*
!rm -rf work/mp4_img/* work/mp4_img2/* work/img2video/* work/img/*
!mkdir work/mp4_img work/mp4_img2 work/img2video work/img

mkdir: cannot create directory ‘work/mp4_img’: File exists
mkdir: cannot create directory ‘work/mp4_img2’: File exists
mkdir: cannot create directory ‘work/img2video’: File exists
mkdir: cannot create directory ‘work/img’: File exists


# 3、设置GPU环境和参数

In [11]:
%env CUDA_VISIBLE_DEVICES=0
%matplotlib inline

env: CUDA_VISIBLE_DEVICES=0


In [10]:
img="person.jpg"#所需处理的图片
text="Happy new year!\nGive me a big red envelope!"#需要转语音的文本,支持多行文本

# 4、利用Fastspeech将文字转语音

In [ ]:
def writetext(text):
    a=open('article.txt', 'w')
    a.write(text)
    a.close()
writetext(text)

In [ ]:
%cd /home/aistudio/Parakeet/examples/fastspeech

/home/aistudio/Parakeet/examples/fastspeech


In [ ]:
%run synthesis.py \
    --use_gpu=1 \
    --alpha=1.0 \
    --checkpoint='./fastspeech_ljspeech_ckpt_1.0/step-162000' \
    --config='./fastspeech_ljspeech_ckpt_1.0/ljspeech.yaml' \
    --output='./synthesis' \
    --vocoder='waveflow' \
    --config_vocoder='./waveflow_res128_ljspeech_ckpt_1.0/waveflow_ljspeech.yaml' \
    --checkpoint_vocoder='./waveflow_res128_ljspeech_ckpt_1.0/step-2000000' \
    --text_input='/home/aistudio/article.txt'

{'alpha': 1.0,
 'checkpoint': './fastspeech_ljspeech_ckpt_1.0/step-162000',
 'checkpoint_vocoder': './waveflow_res128_ljspeech_ckpt_1.0/step-2000000',
 'config': './fastspeech_ljspeech_ckpt_1.0/ljspeech.yaml',
 'config_vocoder': './waveflow_res128_ljspeech_ckpt_1.0/waveflow_ljspeech.yaml',
 'output': './synthesis',
 'text_input': '/home/aistudio/article.txt',
 'use_gpu': 1,
 'vocoder': 'waveflow'}
[checkpoint] Rank 0: loaded model from ./fastspeech_ljspeech_ckpt_1.0/step-162000.pdparams
[checkpoint] Rank 0: loaded model from ./waveflow_res128_ljspeech_ckpt_1.0/step-2000000.pdparams
Synthesis completed !!!
[checkpoint] Rank 0: loaded model from ./waveflow_res128_ljspeech_ckpt_1.0/step-2000000.pdparams
Synthesis completed !!!


In [ ]:
# 生成list文件
for i,line in enumerate(open('/home/aistudio/article.txt')): 
    with open('waveflow_article3.txt', 'a') as f:
        result = 'file synthesis/samples/waveflow' + str(i) +'.wav'
        f.write(result + "\n")

In [ ]:
# 音频拼接
!ffmpeg -f concat -i waveflow_article3.txt -c copy 'waveflow_article3.wav'

ffmpeg version 2.8.15-0ubuntu0.16.04.1 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 5.4.0 (Ubuntu 5.4.0-6ubuntu1~16.04.10) 20160609
  configuration: --prefix=/usr --extra-version=0ubuntu0.16.04.1 --build-suffix=-ffmpeg --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --cc=cc --cxx=g++ --enable-gpl --enable-shared --disable-stripping --disable-decoder=libopenjpeg --disable-decoder=libschroedinger --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmodplug --enable-libmp3lame --enable-libopenjpeg --enable-libopus --enable-libpulse --enable-librtmp --enable-libschroedinger --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --

# 5、利用Wav2lip将图片转换为视频并唇形合成

In [12]:
%cd /home/aistudio

/home/aistudio


In [2]:
#!mkdir sndfile
%cd sndfile
#!wget http://www.mega-nerd.com/libsndfile/files/libsndfile-1.0.28.tar.gz
#!tar xzvf libsndfile-1.0.28.tar.gz

%cd libsndfile-1.0.28
!./configure --prefix=/home/aistudio/build_libs CFLAGS=-fPIC --enable-shared 
!make
!make install

/home/aistudio/sndfile
/home/aistudio/sndfile/libsndfile-1.0.28
checking build system type... x86_64-pc-linux-gnu
checking host system type... x86_64-pc-linux-gnu
checking target system type... x86_64-pc-linux-gnu
checking for a BSD-compatible install... /usr/bin/install -c
checking whether build environment is sane... yes
checking for a thread-safe mkdir -p... /bin/mkdir -p
checking for gawk... no
checking for mawk... mawk
checking whether make sets $(MAKE)... yes
checking whether make supports nested variables... yes
checking whether make supports nested variables... (cached) yes
checking for style of include used by make... GNU
checking for gcc... gcc
checking whether the C compiler works... yes
checking for C compiler default output file name... a.out
checking for suffix of executables... 
checking whether we are cross compiling... no
checking for suffix of object files... o
checking whether we are using the GNU C compiler... yes
checking whether gcc accepts -g... yes
checking for 

In [13]:
%cd ~

/home/aistudio


In [4]:
!tar -xf /home/aistudio/PaddleGAN.tar
!pip install -r PaddleGAN/requirements.txt

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 2.5MB 24.5MB/s eta 0:00:01
ERROR: parakeet 0.0.0 has requirement numba==0.47.0, but you'll have numba 0.48.0 which is incompatible.


In [8]:
def combine_image_to_video(fps, count ,is_print=False):
    '''
        合并图像到视频
    '''
    comb_path="../aistudio/work/img/"
    output_file_path="../aistudio/work/img2video/video.mp4"
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')    
    
    file_items = os.listdir(comb_path)
    file_len = len(file_items)
    # print(comb_path, file_items)
    if file_len > 0 :
        temp_img = cv2.imread(os.path.join(comb_path, file_items[0]))
        img_height, img_width = temp_img.shape[0], temp_img.shape[1]
        
        out = cv2.VideoWriter(output_file_path, fourcc, fps, (img_width, img_height))

        for i in range(count):
            pic_name = os.path.join(comb_path+"img.jpg")
            img = cv2.imread(pic_name)
            out.write(img)
        out.release()

In [14]:
import os
import cv2
import librosa
def Wav2lip(img):
    filepath = 'Parakeet/examples/fastspeech/waveflow_article3.wav'
    duration=librosa.get_duration(filename=filepath)
    command0="cp "+img+" work/img/img.jpg"
    os.system(command0)
    #videopath="test.mp4"
    #video_capture = cv2.VideoCapture(videopath)
    #fps = video_capture.get(cv2.CAP_PROP_FPS)
    fps=30
    #count=0
    #while(True):
    #    ret, frame = video_capture.read() 
    #    if ret:
    #        count += 1
    #    else:
    #        break
    #print(fps,int(duration*fps))
    #video_capture.release()
    combine_image_to_video(fps,int(duration*fps))
    #command1="export PYTHONPATH=$PYTHONPATH:PaddleGAN && python PaddleGAN/applications/tools/wav2lip.py --face work/img2video/video.mp4 --audio "+filepath+" --outfile work/img2video/lip.mp4"
    os.system("export PYTHONPATH=$PYTHONPATH:/home/aistudio/PaddleGAN && python /home/aistudio/PaddleGAN/applications/tools/wav2lip.py --face /home/aistudio/work/img2video/video.mp4 --audio /home/aistudio/Parakeet/examples/fastspeech/waveflow_article3.wav --outfile /home/aistudio/work/img2video/video2.mp4")

Wav2lip(img)

# 6、利用humanseg进行人像分割

In [15]:
!pip install paddlehub -U -i https://pypi.tuna.tsinghua.edu.cn/simple #用了清华的镜像源

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Requirement already up-to-date: paddlehub in /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages (2.0.0)


In [16]:
! hub install humanseg_server==1.2.1

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matp

In [17]:
import cv2
import numpy as np
import paddlehub as hub

human_seg = hub.Module(name='humanseg_server')
cap_video = cv2.VideoCapture('/home/aistudio/work/img2video/video2.mp4')
fps = cap_video.get(cv2.CAP_PROP_FPS)
save_path = '/home/aistudio/work/img2video/video3.mp4'
width = int(cap_video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap_video.get(cv2.CAP_PROP_FRAME_HEIGHT))
cap_out = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, (width, height))
prev_gray = None
prev_cfd = None
while cap_video.isOpened():
    ret, frame_org = cap_video.read()
    if ret:
        [img_matting, prev_gray, prev_cfd] = human_seg.video_stream_segment(frame_org=frame_org, frame_id=cap_video.get(1), prev_gray=prev_gray, prev_cfd=prev_cfd)
        img_matting = np.repeat(img_matting[:, :, np.newaxis], 3, axis=2)
        bg_im = np.ones_like(img_matting) * 255
        comb = (img_matting * frame_org + (1 - img_matting) * bg_im).astype(np.uint8)
        cap_out.write(comb)
    else:
        break

cap_video.release()
cap_out.release()

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):
[2021-02-18 19:33:15,556] [ WARNING] - The _initialize method in HubModule will soon be deprecated, you can use the __init__() to handle the initialization of the object


**注意此处有PaddleHub的BUG，需要把上面这一块代码运行多一次即可**

`AttributeError: module has no attribute '__name__'`
这个问题的原因是.pyc文件的报错。

pyc文件，是python编译后的字节码（bytecode）文件。

.pyc文件作用：经过python解释器，会生成机器码运行（这也是为什么pyc文件可以跨平台部署，类似于java的跨平台，java中JVM运行的字节码文件）。下次调用直接调用pyc，而不调用py文件。直到你这个py文件有改变。python解释器会检查pyc文件中的生成时间，对比py文件的修改时间，如果py更新，那么就生成新的pyc。

综上所述，报错与代码无关，跑多一次就OK了!

# 7、将视频转为GIF

In [ ]:
import moviepy.editor as mpy
content = mpy.VideoFileClip("/home/aistudio/work/img2video/video3.mp4")
content.write_gif("/home/aistudio/work/img2video/final.gif")

MoviePy - Building file /home/aistudio/work/img2video/final.gif with imageio.


t:   0%|          | 0/91 [00:00<?, ?it/s, now=None]t:  78%|███████▊  | 71/91 [00:06<00:01, 11.22it/s, now=None]

#  8、发红包

1、将GIF图下载到本地，并发送给自己手机

![](https://ai-studio-static-online.cdn.bcebos.com/ee3db54243024b1ca4506755000e97db8aa266f6fed44c4a96983129914c4e1b)

2、将GIF添加到自己的表情库里

![](https://ai-studio-static-online.cdn.bcebos.com/015f0582e0c9462689b14d877ffb05d20bb019ddd7e841969270c247466a5492)

3、打开发红包页面，点击笑脸

![](https://ai-studio-static-online.cdn.bcebos.com/0fd8ba32cc4c498c94d3fa52ec34f6b3cd1e84402675446ebf1599047b3723cb)

![](https://ai-studio-static-online.cdn.bcebos.com/bf5403a2b06942edbbdc8bb532034ea0926f156d529b406ea03a8478cfbbb3bf)

4、完成，并看看效果吧

![](https://ai-studio-static-online.cdn.bcebos.com/3f0eca7ce99245a2b503d0415a4bb143efe51383d51a4c769615dc2cc055f227)



# 关于作者

😃姓名：曾焯淇😃

😃学历：高中😃

😃From：广东 佛山（欢迎面基）😃

我在AI Studio上获得黄金等级，点亮3个徽章，来互关呀~ https://aistudio.baidu.com/aistudio/personalcenter/thirdview/233221